In [1]:
import graphsage_experiments
from torch_geometric.datasets import Reddit
import torch.nn.functional as F
import graph_information

# Download dataset

In [2]:
dataset = Reddit("./")
data = dataset[0]

# Check details from dataset 

In [4]:
graph_information.visualize_information_graph(dataset)

Dataset: Reddit()
-------------------
Number of graphs: 1
Number of nodes: 232965
Number of features: 602
Number of classes: 41

Graph:
------
Training nodes: 153431
Evaluation nodes: 23831
Test nodes: 55703
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False


In [ ]:
learning_rate = 0.0001 # variable to change/play around with for experiments --> 0.0001
aggregator = 'mean' # variable to change/play around with for experiments

epochs = 10
dropout_rate = 0.4
normalization = True 
activation_function = F.relu
bias = True
batch_size =  512
neighborhood_1 = 25
neighborhood_2 = 10
embedding_dimension = 128
hidden_layer = 512

In [ ]:
number_features, number_nodes = data.num_features, data.x.shape[0]

In [ ]:
emb_matrix = graphsage_experiments.compute_embedding_matrix(
    data = data,
    number_features = number_features,
    number_nodes = number_nodes,
    batch_size = batch_size,
    hidden_layer = hidden_layer, 
    epochs = epochs, 
    neighborhood_1 = neighborhood_1,
    neighborhood_2 = neighborhood_2,
    embedding_dimension = embedding_dimension,
    learning_rate = learning_rate,
    dropout_rate = dropout_rate,
    activation_function = activation_function,
    aggregator = aggregator,
    activation_before_normalization= True,
    bias= True,
    normalize = normalization,
)
